In [12]:
import pandas as pd
import numpy as np

import tarfile
import shutil
import urllib
import os
import copy

import hashlib

import random
from itertools import chain
from typing import Any, Tuple
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
import torchvision
from torchvision import transforms
import torchvision.transforms.v2 as transforms2
from PIL import Image
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from tabulate import tabulate
from pathlib import Path

import warnings

In [13]:
class BasicDatasetCIFAR100():
    """
    Contains the main Dataset with GT Label and Expert Label for every Image, sorted by file name
    """
    def __init__(self, path_labels, path_data):

        self.path_labels = path_labels
        self.cifar_dataset = self.load_cifar_100(path_data)
        self.gt_df = pd.DataFrame({"GT": self.cifar_dataset.targets})
        self.gt_df = self.gt_df.rename_axis("Image ID").reset_index()

        print("Number of images of the whole dataset: " + str(len(self.gt_df["Image ID"].values)))
        
    def load_cifar_100(self, path):
        """
        Loads the CIFAR100 dataset from pytorch
        """
        transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        return torchvision.datasets.CIFAR100(root=f'{path}/CIFAR100', train=True, download=True, transform=transform)
        

    def get_synthetic_labels(self, path, experts_args, seed):
        expert_labels = {}
        for name, expert_arg in experts_args.items():
            path_name = self. expert_to_path(name, expert_arg.copy(), seed)
            if self.check_if_expert_exists(path, path_name, expert_arg["strength"]):
                expert_labels[seed][name] == self.load_expert_from_path(pat, path_name, expert_arg["strength"])
            else:
                expert_labels[seed][name] == self.create_synthetic_labels(expert_args, name, seed)["train"]
             
        return self.synthetic_labels

    def create_synthetic_labels(self, expert_args, name, seed):
        expert_labels = synex.generate_synthetic_expert(strength=expert_arg["strength"], binary=expert_arg["binary"], num_classes=expert_arg["num_classes"], per_s=expert_arg["per_s"], per_w=expert_arg["per_w"], seed=seed, path=self.path_labels, name=name)
        return expert_labels

    def check_if_expert_exists(self, path, name_path, strength):
        my_file = Path(f'{path}/synthetic_experts/{path_name}/cifar100_expert_{strength}_labels.json')
        return my_file.is_file()

    def load_expert_from_path(self, path, path_name, strength):
        with open(f'{path}/synthetic_experts/{path_name}/cifar100_expert_{strength}_labels.json', 'r') as f:
            labels = json.load(f)
        return labels["train"]

    def expert_to_path(self, name, args, seed):
        args = args.copy()
        args["name"] = name
        args["seed"] = seed
        return self.param_to_path(args)

    def param_to_path(self, args):
        s = ""
        for key, value in args.items():
            s += f"{key}_{value}_"
        s = s[:-1]
        return int(hashlib.sha256(s.encode('utf-8')).hexdigest(), 16) % 10**8
    

    def getExpert(self, id):
        """
        Returns the data for the given expert
        """
        return self.data[["Image ID", "GT", str(id)]].copy()

    def getData(self):
        """
        Returns all data
        """
        return self.data.copy()
    
    def getDataForLabelers(self, labelerIds):
        """
        Returns the data with ["Image ID", "GT", [labelerIds]]
        """
        temp = self.data[["Image ID", "GT"]].copy()
        for labelerId in labelerIds:
            temp[str(labelerId)] = self.data[str(labelerId)]
        return temp

In [14]:
bd = BasicDatasetCIFAR100(path_labels="../../../Datasets/CIFAR100", path_data="../../../Datasets/CIFAR100")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 169001437/169001437 [00:07<00:00, 24062902.82it/s]


Extracting ../../../Datasets/CIFAR100/CIFAR100/cifar-100-python.tar.gz to ../../../Datasets/CIFAR100/CIFAR100
Number of images of the whole dataset: 50000


In [7]:
bd.get_synthetic_labels()

TrueEx_cifar100_60_labels.json
